In [5]:
import cv2

def find_eye_pairs(eyes, max_vertical_diff=20, max_horizontal_gap=150):
    """
    Find pairs of eyes from the list of eye bounding boxes.

    Args:
        eyes: list of (x, y, w, h)
        max_vertical_diff: max allowed vertical difference between eyes to be a pair
        max_horizontal_gap: max allowed horizontal distance between eyes to be a pair

    Returns:
        List of pairs [(eye1, eye2), ...]
    """
    pairs = []
    eyes = sorted(eyes, key=lambda e: e[0])  # sort by x (left to right)

    for i in range(len(eyes)):
        for j in range(i+1, len(eyes)):
            x1, y1, w1, h1 = eyes[i]
            x2, y2, w2, h2 = eyes[j]

            vertical_diff = abs(y1 - y2)
            horizontal_gap = abs((x2 + w2/2) - (x1 + w1/2))

            # Check vertical alignment and reasonable horizontal gap
            if vertical_diff <= max_vertical_diff and horizontal_gap <= max_horizontal_gap:
                pairs.append((eyes[i], eyes[j]))
    return pairs


eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.15, minNeighbors=8)

    pairs = find_eye_pairs(eyes)

    if pairs:
        # Find the biggest pair by combined width
        biggest_pair = max(pairs, key=lambda pair: pair[0][2] + pair[1][2])

        for (x, y, w, h) in biggest_pair:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.putText(frame, 'Biggest Eye Pair', (biggest_pair[0][0], biggest_pair[0][1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        cv2.putText(frame, 'No Eye Pairs Detected', (20, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow('Biggest Eye Pair Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()